In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/09 18:20:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/09 18:20:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
case = spark.read.csv("data/case.csv", sep=",",
                      header=True, inferSchema=True)
dept = spark.read.csv("data/dept.csv", sep=",",
                      header=True, inferSchema=True)
src = spark.read.csv("data/source.csv", sep=",",
                     header=True, inferSchema=True)

In [3]:
#create subdirectory with data and _SUCCESS text file
src.write.json("data/source_json", mode="overwrite")
src.write.csv("data/source_csv", mode="overwrite")

In [13]:
case.printSchema()
dept.printSchema()
src.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)




    How old is the latest (in terms of days past SLA) currently open issue?
    
    How long has the oldest (in terms of days since opened) currently opened issue been open?
    
    How many Stray Animal cases are there?

    How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

    Convert the council_district column to a string column.
    
    Extract the year from the case_closed_date column.

    Convert num_days_late from days to hours in new columns num_hours_late.

    Join the case data with the source and department data.

    Are there any cases that do not have a request source?

    What are the top 10 service request types in terms of number of requests?
    
    What are the top 10 service request types in terms of average days late?
    
    Does number of days late depend on department?
    
    How do number of days late depend on department and request type?


In [104]:
#join
case = case.join(src,
               on = case.source_id==src.source_id,
               how = 'left')\
            .join(dept,
                  on = case.dept_division==dept.dept_division,
                  how = 'left')\
            .drop(case.source_id).drop(case.dept_division)

In [105]:
#reformat dates
fmt = "M/d/yy H:mm"
case = case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))\
            .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))\
            .withColumn("SLA_due_date", to_timestamp("SLA_due_date", fmt))

In [106]:
#add columns
case = case.withColumn("council_district", col("council_district").cast("string"))\
            .withColumn('year', year('case_closed_date'))\
            .withColumn("num_hours_late", expr("num_days_late * 24 AS num_hours_late"))

In [107]:
case.show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 case_id                | 1014127332                           
 case_opened_date       | 2018-01-01 00:42:00                  
 case_closed_date       | 2018-01-01 12:29:00                  
 SLA_due_date           | 2020-09-26 00:42:00                  
 case_late              | NO                                   
 num_days_late          | -998.5087616000001                   
 case_closed            | YES                                  
 service_request_type   | Stray Animal                         
 SLA_days               | 999.0                                
 case_status            | Closed                               
 request_address        | 2315  EL PASO ST, San Antonio, 78207 
 council_district       | 5                                    
 source_id              | svcCRMLS                             
 source_username        | svcCRMLS                             
 dept_division          | Field Operatio

In [57]:
open_ = case.filter(expr('case_closed=="NO"'))\
             .withColumn('days_past_SLA', datediff(current_timestamp(), 'SLA_due_date'))\
             .withColumn('duration', datediff(current_timestamp(), 'case_opened_date'))

In [62]:
#latest issue
open_.select(min('days_past_SLA')).show()
#oldest issue
open_.select(max('duration')).show()

+------------------+
|min(days_past_SLA)|
+------------------+
|              -162|
+------------------+



+-------------+
|max(duration)|
+-------------+
|         1407|
+-------------+



In [64]:
#strays
case.where(expr('service_request_type=="Stray Animal"')).count()

9325

In [63]:
#field operations and not officer standby
case.where(expr('dept_division=="Field Operations"'))\
    .where(expr('service_request_type!="Officer Standby"')).count()

39810

In [84]:
df.show(1, vertical=True)

-RECORD 0--------------------------------------
 case_id                | 1014127332           
 case_opened_date       | 2018-01-01 00:42:00  
 case_closed_date       | 2018-01-01 12:29:00  
 SLA_due_date           | 2020-09-26 00:42:00  
 case_late              | NO                   
 num_days_late          | -998.5087616000001   
 case_closed            | YES                  
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 request_address        | 2315  EL PASO ST,... 
 council_district       | 5                    
 num_hours_late         | -23964.2102784       
 source_id              | svcCRMLS             
 source_username        | svcCRMLS             
 dept_division          | Field Operations     
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES                  
only showing top 1 row



In [88]:
#null username
df.filter("source_username is null").show(vertical=True)

(0 rows)



In [90]:
#ten most popular services
df.groupby('service_request_type').count().sort(col('count').desc()).show(10)
#ten most tardy services
df.where('case_late=="YES"').groupBy('service_request_type')\
            .agg(mean('num_days_late')).sort(desc('avg(num_days_late)')).show(10, truncate=False)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|27415|
|Overgrown Yard/Trash|21448|
|Cart Exchange Req...|12815|
|        Damaged Cart|10262|
|        Bandit Signs| 9741|
|        Stray Animal| 9604|
|Aggressive Animal...| 8976|
|Front Or Side Yar...| 8953|
|     Pot Hole Repair| 8145|
|Junk Vehicle On P...| 7243|
+--------------------+-----+
only showing top 10 rows



+------------------------------+------------------+
|service_request_type          |avg(num_days_late)|
+------------------------------+------------------+
|Traffic Sign Graffiti         |167.64583330000002|
|Zoning: Junk Yards            |134.49897372428572|
|Status: On-Going CIMS Projects|114.73973603267096|
|Zoning: Recycle Yard          |109.63640396651162|
|Complaint                     |106.29415441909525|
|Curbstoning                   |85.67311176816668 |
|Boarding Home Inspection      |84.38680411125    |
|Vendors                       |75.36385313544446 |
|Reservation Assistance        |66.03116319       |
|"Vacant Lot-State (12"")"     |66.01824074       |
+------------------------------+------------------+
only showing top 10 rows



In [91]:
#tardiness by dept
df.where('case_late=="YES"').groupBy('standardized_dept_name')\
    .agg(mean('num_days_late')).sort(desc('avg(num_days_late)')).show(truncate=False)

+------------------------+------------------+
|standardized_dept_name  |avg(num_days_late)|
+------------------------+------------------+
|Customer Service        |106.18038971065275|
|DSD/Code Enforcement    |36.25423030779672 |
|Parks & Recreation      |23.550176549963236|
|Trans & Cap Improvements|15.815148058263395|
|Animal Care Services    |15.480924657469522|
|Solid Waste             |7.454184042992661 |
|Metro Health            |4.339849306217296 |
+------------------------+------------------+



In [92]:
#tardiness by dept and request type
df.where('case_late=="YES"').groupBy('standardized_dept_name', 'service_request_type')\
    .agg(mean('num_days_late')).sort(desc('avg(num_days_late)')).show(truncate=False)

+------------------------+----------------------------------------+------------------+
|standardized_dept_name  |service_request_type                    |avg(num_days_late)|
+------------------------+----------------------------------------+------------------+
|Trans & Cap Improvements|Traffic Sign Graffiti                   |167.64583330000002|
|DSD/Code Enforcement    |Zoning: Junk Yards                      |134.49897372428572|
|Trans & Cap Improvements|Status: On-Going CIMS Projects          |114.73973603267096|
|DSD/Code Enforcement    |Zoning: Recycle Yard                    |109.63640396651162|
|Customer Service        |Complaint                               |106.29415441909525|
|DSD/Code Enforcement    |Curbstoning                             |85.67311176816668 |
|DSD/Code Enforcement    |Boarding Home Inspection                |84.38680411125    |
|DSD/Code Enforcement    |Vendors                                 |75.36385313544446 |
|Parks & Recreation      |Reservation Assis